In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from bayes_opt import BayesianOptimization

data = pd.read_excel('3.xlsx')
x = data.iloc[:, :2].values  
y = data.iloc[:, -1].values  

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.1, random_state=42)

def optimize(max_depth, min_samples_split, min_samples_leaf):
    kflod = KFlod(n_splits=10, shuffle=True, random_state=42)
    model = DecisionTreeRegressor(max_depth=int(max_depth),
                                  min_samples_split=int(min_samples_split),
                                  min_samples_leaf=int(min_samples_leaf),
                                  random_state=42)
    model.fit(train_x, train_y)
    predictions = model.predict(test_x)
    r2 = r2_score(test_y, predictions)
    return r2

optimizer = BayesianOptimization(
    f=optimize,
    pbounds={
        'max_depth': (1, 55),
        'min_samples_split': (2, 50),
        'min_samples_leaf': (1, 50),
    },
    random_state=42,
)

optimizer.maximize(n_iter=100000, init_points=50)

best_params = optimizer.max['params']
best_params['max_depth'] = int(best_params['max_depth'])
best_params['min_samples_split'] = int(best_params['min_samples_split'])
best_params['min_samples_leaf'] = int(best_params['min_samples_leaf'])

best_model = DecisionTreeRegressor(**best_params, random_state=42)
best_model.fit(train_x, train_y)
train_pred = best_model.predict(train_x)
test_pred = best_model.predict(test_x)

train_mse = mean_squared_error(train_y, train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(train_y, train_pred)
train_r2 = r2_score(train_y, train_pred)

test_mse = mean_squared_error(test_y, test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(test_y, test_pred)
test_r2 = r2_score(test_y, test_pred)

print(f'Train - MSE: {train_mse}, RMSE: {train_rmse}, MAE: {train_mae}, R^2 score: {train_r2}')
print(f'Test - MSE: {test_mse}, RMSE: {test_rmse}, MAE: {test_mae}, R^2 score: {test_r2}')